O gráfico de áreas empilhadas deverá ter um filtro para mudarmos a escala da visualização (linear ou logarítmica). Além disso devemos permitir a métrica à ser analisada.

Usaremos essa visualização para observar a evolução da soma de uma métrica por dia.

In [1]:
import plotly

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

modes = ['linear', 'log10']
metrics = ['views', 'likes', 'dislikes', 'comment_count']

## Lendo os dados

In [2]:
us_data = pd.read_csv('USdata.csv', parse_dates=['publish_time', 'trending_date'])

grouped_data = us_data.groupby(['trending_date','category_name'])[metrics]
grouped_data = grouped_data.sum()
grouped_data = grouped_data.reset_index()

grouped_data.head()

,trending_date,category_name,views,likes,dislikes,comment_count
0,2017-11-14,Autos & Vehicles,814856,20498,728,1350
1,2017-11-14,Comedy,16643490,860087,22772,54405
2,2017-11-14,Education,966237,45209,890,7305
3,2017-11-14,Entertainment,28047357,858276,42437,105286
4,2017-11-14,Film & Animation,6015279,92770,5994,12492


## Definindo espectro de cores

In [3]:
# Gerando um array de cores do arco-íris fixando a saturação e brilho da
# representação HSL da cor e caminhando pela matiz.
category_colors = [f'hsl({h},50%,50%)' for h in np.linspace(0, 360, 30)]

## Construíndo a visualização

In [4]:
def get_visibility(idx, n=8, n_categories=16):
    """ Função auxiliar para retornar uma lista de visibilidade para os filtros """
    
    visibility = []
    booleans = [(i == idx) for i in range(n)]
    
    for b in booleans:
        visibility.extend([b] * n_categories)
    
    return visibility

In [9]:
## Criando trace para cada plot (serão 8 pares (mode, metric) no total)
traces = []
categories = sorted(us_data['category_name'].unique())

for mode in modes:
    for metric in metrics:
        for i in range(len(categories)):
            data = grouped_data[grouped_data['category_name'] == categories[i]]
            traces.append(go.Bar(
                name = categories[i],
                marker_color = category_colors[i],
                x = data['trending_date'],
                y = data[metric] if mode == 'linear' else np.log10(1+data[metric]),
                visible = True if mode == 'linear' and metric == 'views' else False
            ))
            
fig = go.Figure(data=traces[::-1])


## Definindo botões para filtro 
idx = 0    
buttons = []

for mode in modes:
    for metric in metrics:
        buttons.append({
            'label': f'{metric} - {mode}',
            'method': 'update',
            'args': [{'visible': get_visibility(idx)}]
        })
        
        idx += 1

fig.update_layout(updatemenus=[
    dict(
        x = -0.15,
        active = 0,
        showactive = True,
        buttons = buttons
    )
])

## Definindo outras propriedades de layout
fig.update_layout(
    title='Somatório diário das métricas por categoria',
    xaxis_title='trending date',
    barmode='stack'
)

fig.show()